In [210]:
%load_ext autoreload
%autoreload 2
from models import *
from names import *
import seaborn as sns
import matplotlib.pyplot as plt
import cairosvg


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [211]:
results_df = aggregate_results_from_subfolders(".")

results_df.sort_values(by="difference of ACE", ascending=True, inplace=True)
# results_df.sort_values(by="pre_r_squared test", ascending=False, inplace=True)

results_df.to_csv("results.csv", index=False)
print(f"{len(results_df)} records")

63 records


In [212]:
# results_digest = results_df[
#     [
#         "precovid ACE",
#         "postcovid ACE",
#         "difference of ACE",
#         "outcome",
#         "treatment",
#     ]
# ].copy()
results_digest = results_df[
    [
        "difference of ACE",
        "outcome",
        "treatment",
    ]
].copy()
results_digest["outcome description"] = results_digest["outcome"].apply(
    lambda x: full_dictionary[x]
)
results_digest["treatment description"] = results_df["treatment"].apply(
    lambda x: full_dictionary[x]
)
result_digest = results_digest[
    ["treatment description", "outcome description", "difference of ACE"]
]

In [213]:
len(results_digest["outcome"].unique())

7

In [214]:
len(results_digest["treatment"].unique())

14

In [215]:
df_wide = results_digest.pivot(
    index="treatment description",
    columns="outcome description",
    values="difference of ACE",
)
df_wide

outcome description,gad2_score,pam,phq2_score,phq4_score,social_level,sse_score,stress
treatment description,,,,,,,
At home,-0.0668,0.628,-0.1590,-0.29500,NaN,-0.0481,-0.07430
Distance traveled,NaN,NaN,NaN,NaN,-0.13500,NaN,NaN
Duration unlocked phone in social loc.,0.2690,-0.570,0.0815,0.38000,NaN,-0.8750,0.18900
Exercise,-0.0164,NaN,NaN,0.00335,0.16000,0.2230,-0.00736
Frequency of unlocked phone in social loc.,-0.1610,NaN,0.0687,0.09070,-0.09350,0.2950,-0.07460
Motion at social locations,-0.1740,NaN,NaN,NaN,0.00667,NaN,NaN
Sports,-0.0396,-0.230,-0.2090,-0.22100,0.06280,0.7560,-0.04310
Studying,-0.0918,-0.386,NaN,-0.05260,0.15600,-0.3640,NaN
Time in social loc.,0.0838,0.011,-0.2480,-0.10700,0.09540,-1.1500,0.14000


In [216]:
import pdfkit


def color_gradient(val):
    # Normalize the value to the range [0, 1]
    norm = (val - df_wide.min().min()) / (
        df_wide.max().max() - df_wide.min().min()
    )
    # Use a colormap to get the color
    cmap = plt.get_cmap("coolwarm")
    # Get the color as a tuple (r, g, b, a)
    color = cmap(norm)
    return (
        f"background-color: rgba({int(color[0]*255)}, {int(color[1]*255)},"
        f"{int(color[2]*255)}, {color[3]}); color: black; "
        "font-family: Arial; font-size: 12px;font-weight: bold;"
    )


styled_df = df_wide.style.map(color_gradient)
# Replace NaN with an empty string
styled_df = styled_df.format(na_rep="")

# Add borders to the cells
styled_df = styled_df.set_table_styles(
    [{"selector": "th, td", "props": [("border", "1px solid black")]}]
)
# Convert the styled DataFrame to HTML
html = styled_df.to_html()

# Save the HTML to a file (optional, for debugging)
with open("styled_dataframe.html", "w") as file:
    file.write(html)

options = {
    "disable-smart-shrinking": "",
}

pdfkit.from_string(html, "styled_dataframe.pdf", options=options)

OSError: wkhtmltopdf exited with non-zero code 1. error:
The switch --disable-smart-shrinking, is not support using unpatched qt, and will be ignored.--quiet specified in incorrect location

Name:
  wkhtmltopdf 0.12.6

Synopsis:
  wkhtmltopdf [GLOBAL OPTION]... [OBJECT]... <output file>
  
Document objects:
  wkhtmltopdf is able to put several objects into the output file, an object is
  either a single webpage, a cover webpage or a table of contents.  The objects
  are put into the output document in the order they are specified on the
  command line, options can be specified on a per object basis or in the global
  options area. Options from the Global Options section can only be placed in
  the global options area.

  A page objects puts the content of a single webpage into the output document.

  (page)? <input url/file name> [PAGE OPTION]...
  Options for the page object can be placed in the global options and the page
  options areas. The applicable options can be found in the Pa